<a href="https://colab.research.google.com/github/erdult/LLMFeedNewInfoFromPDF/blob/main/OpenAI_Langchain_pdf_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install langchain
!pip install unstructured # The unstructured library provides open-source components for pre-processing text documents such as PDFs, HTML and Word Documents.
!pip install openai
!pip install pybind11 # pybind11 is a lightweight header-only library that exposes C++ types in Python
!pip install chromadb # the AI-native open-source embedding database
!pip install Cython # Cython is an optimising static compiler for both the Python programming language
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI" # COCO is a large image dataset designed for object detection, segmentation, person keypoints detection, stuff segmentation, and caption generation
!pip install unstructured[local-inference]
!CC=clang CXX=clang++ ARCHFLAGS="-arch x86_64" pip install 'git+https://github.com/facebookresearch/detectron2.git' # Detectron2 is Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms.
!pip install layoutparser[layoutmodels,tesseract] # A Unified Toolkit for Deep Learning Based Document Image Analysis
!pip install pytesseract # Python-tesseract is an optical character recognition (OCR) tool for python.
!pip install Pillow==9.0.0 # The Python Imaging Library adds image processing capabilities to your Python interpreter. Need this version, otherwise errors occur.

  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-install-_yfnsp5c/pycocotools_04736cbe8e374d3b9e4c6e3bb2b8e886
  Running command git clone --filter=blob:none --quiet https://github.com/philferriere/cocoapi.git /tmp/pip-install-_yfnsp5c/pycocotools_04736cbe8e374d3b9e4c6e3bb2b8e886
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=378962 sha256=db228c7cc35785b2d200afcac93f3ed24689ef1cbc49bf21a58867f67089eb25
  Stored in directory: /tmp/pip-ephem-wheel-cache-hvujsvsa/wheels/8e/1a/5f/0c984ae3d65a4d7da4ba3407d2a8d8c8e85dd55f84d4936f04
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.7
    Uninstalling pycocotools-2.0.7:
      Successfully uninstalled pycocotools-2.0.7
ERROR: pip's dependency resolver does not currently tak

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-hmd04vod
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-hmd04vod
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4e80df1e58901557e2824ce3b488d30209a9be33
  Preparing metadata (setup.py) ... done
  Using cached Pillow-9.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 10.0.1
    Uninstalling Pillow-10.0.1:
      Successfully uninstalled Pillow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.10.2 requires Pillow>=9.1, but you have pillow 9.0.0 which is incompatible.


Restart runtime above to get correct libraries imported

In [1]:
import os
os.environ['OPENAI_API_KEY'] = 'OPEN AI API KEY'

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [3]:
from detectron2.config import get_cfg
cfg = get_cfg()
cfg.MODEL.DEVICE = 'cpu' #GPU is recommended

In [4]:
!wget https://s21.q4cdn.com/399680738/files/doc_financials/2022/q4/Meta-12.31.2022-Exhibit-99.1-FINAL.pdf #meta earnings; replace with any pdf

--2023-10-08 14:37:15--  https://s21.q4cdn.com/399680738/files/doc_financials/2022/q4/Meta-12.31.2022-Exhibit-99.1-FINAL.pdf
Resolving s21.q4cdn.com (s21.q4cdn.com)... 68.70.205.1, 68.70.205.4, 68.70.205.3, ...
Connecting to s21.q4cdn.com (s21.q4cdn.com)|68.70.205.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185815 (181K) [application/pdf]
Saving to: ‘Meta-12.31.2022-Exhibit-99.1-FINAL.pdf’

Meta-12.31.2022-Exh 100%[===================>] 181.46K  --.-KB/s    in 0.06s   

2023-10-08 14:37:15 (2.91 MB/s) - ‘Meta-12.31.2022-Exhibit-99.1-FINAL.pdf’ saved [185815/185815]



In [5]:
!mkdir docs
!mv Meta-12.31.2022-Exhibit-99.1-FINAL.pdf docs

mkdir: cannot create directory ‘docs’: File exists


In [6]:
text_folder = 'docs'
loaders = [UnstructuredPDFLoader(os.path.join(text_folder, fn)) for fn in os.listdir(text_folder)]

In [7]:
!apt-get install poppler-utils # error occurs without this, pdf rendering library

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [8]:
!pip install tiktoken

In [9]:
index = VectorstoreIndexCreator().from_loaders(loaders)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


index should be saved so you can use it in the future again

In [10]:
query = "How much revenue did Meta make in 2022?"
index.query(query)

' Meta made $116,609 million in revenue in 2022.'

In [11]:
query = "What are meta's biggest risks?"
index.query(query)

" Meta's biggest risks include macroeconomic conditions, reliance on advertising revenue, dependency on data signals and mobile operating systems, changes to third-party policies, and maintaining and enhancing their brand and reputation."

In [12]:
query = "How much revenue did Meta make in 2022?"
index.query(query)

' Meta made $116,609 million in revenue in 2022.'

In [13]:
query = "What is Goldman sachs CET1 capital in June 2023?"
index.query(query)

" I don't know."

In [16]:
!wget https://www.goldmansachs.com/investor-relations/financials/other-information/2023/2q-pillar3-2023.pdf
!mkdir docs/goldman_financials
!mv 2q-pillar3-2023.pdf docs/goldman_financials
text_folder = 'docs/goldman_financials'
loaders = [UnstructuredPDFLoader(os.path.join(text_folder, fn)) for fn in os.listdir(text_folder)]

--2023-10-08 14:51:16--  https://www.goldmansachs.com/investor-relations/financials/other-information/2023/2q-pillar3-2023.pdf
Resolving www.goldmansachs.com (www.goldmansachs.com)... 23.8.70.33
Connecting to www.goldmansachs.com (www.goldmansachs.com)|23.8.70.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 367583 (359K) [application/pdf]
Saving to: ‘2q-pillar3-2023.pdf’

2q-pillar3-2023.pdf 100%[===================>] 358.97K  --.-KB/s    in 0.05s   

2023-10-08 14:51:16 (7.18 MB/s) - ‘2q-pillar3-2023.pdf’ saved [367583/367583]



In [18]:
index = VectorstoreIndexCreator().from_loaders(loaders)

In [19]:
query = "What is Goldman sachs CET1 capital in June 2023?"
index.query(query)

' 98,188 million'